<a href="https://colab.research.google.com/github/FranciscoChavezData/Data-Analytics-IDS-Fase4/blob/main/Desbalance_de_Clases_y_SMOTE_datafase4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paso 1: Configuración del Entorno (Google Colab)

**1.1 Importación de Librerías**

In [1]:
# BITÁCORA: PASO 1.1 - Importación de Librerías Necesarias
# Pandas: Para manejar DataFrames (tablas).
# Scikit-learn: Librería principal para Machine Learning (IA).

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from IPython.display import display
from google.colab import drive # Necesario para montar tu Drive

**1.2 Montar Google Drive**

In [2]:
# BITÁCORA: PASO 1.2  - Montar Google Drive
# Ejecuta esta celda y sigue el enlace para dar permiso a Colab de acceder a tu Drive.
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive montado con éxito. Puedes encontrar tus archivos en la carpeta '/content/drive/MyDrive'.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive montado con éxito. Puedes encontrar tus archivos en la carpeta '/content/drive/MyDrive'.


# **ESTE PASO REEMPLAZA LOS PASOS 2, 3 Y 4 FRAGMENTADOS.**
En este nuevo paso cargamos el Código Monolítico (El Nuevo Paso 2/3/4)

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
from IPython.display import display

print("---------------------------------------------------------")
print("INICIO DEL PROCESO MONOLÍTICO: Limpieza, Escalado y SMOTE.")
print("---------------------------------------------------------")

# --- 1. CARGA DE DATOS ---
# ASEGÚRATE DE QUE ESTA RUTA SEA CORRECTA
file_name = '/content/drive/MyDrive/Colab Notebooks/DataAnalytics/iotsim-hydraulic-system-2.csv'
print(f"1. Cargando archivo: {file_name}...")

# Usamos low_memory=False para manejar el archivo grande de 2.1M de registros.
df = pd.read_csv(file_name, low_memory=False)
print(f"✅ Archivo cargado con éxito. Total de filas: {df.shape[0]}")

# Definición de X (features) y y (target/etiqueta)
X = df.drop('label', axis=1).copy()
y = df['label'].copy()

# --- 2. ENCODIFICACIÓN DE LA ETIQUETA 'y' ---
# Convertimos todas las etiquetas (Mirai, Benign, etc.) a números (0, 1, 2...)
print("\n2. Codificación de etiquetas (LabelEncoder)...")
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y = pd.Series(y_encoded, index=y.index, name='label_encoded')

# --- 3. LIMPIEZA Y CONVERSIÓN DE CARACTERÍSTICAS 'X' ---
print("3. Limpieza y Procesamiento de X...")
# Columnas que históricamente causan problemas o son redundantes
columns_to_drop_manual = [
    'frame.time', 'ip.dst', 'ip.src', 'eth.src', 'eth.dst', 'ip.checksum',
    'tcp.options', 'tcp.pdu.size', 'udp.srcport', 'udp.dstport', 'tcp.checksum',
    'tcp.flags', 'ip.flags', 'ip.tos', 'frame.protocols'
]
columns_to_drop_existing = [col for col in X.columns if col in columns_to_drop_manual]
X_processed = X.drop(columns=columns_to_drop_existing, axis=1, errors='ignore').copy()

# Convertir columnas de texto restantes (si las hay) a numérico (NaN si falla)
for col in X_processed.columns:
    if X_processed[col].dtype == 'object':
        X_processed[col] = pd.to_numeric(X_processed[col], errors='coerce')

# Imputar/Rellenar valores faltantes (NaN) con la media
X_processed = X_processed.fillna(X_processed.mean(numeric_only=True))
print("✅ Limpieza y conversión completada.")


# --- 4. DIVISIÓN (SPLIT) Y ESCALADO (SCALE) ---
print("\n4. División y Escalado de Datos...")
# Split con stratify para mantener la proporción de clases en train y test
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y, test_size=0.2, random_state=42, stratify=y
)

# Escalado con StandardScaler (solo en columnas numéricas)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.values)
X_test_scaled = scaler.transform(X_test.values)

X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
print("✅ Split y Escalado completados.")

# --- 5. APLICANDO SMOTE (BALANCEO) ---
print("\n5. Aplicando SMOTE (Balanceo) para la clase minoritaria 'Benign'...")
# SMOTE solo se aplica al conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

print("✅ SMOTE aplicado con éxito.")
print("\nDistribución de Clases DESPUÉS del balanceo (y_train_res):")
# Mostramos el balanceo. Las clases minoritarias deben tener el mismo conteo que la clase mayoritaria.
display(y_train_res.value_counts())

print("\n---------------------------------------------------------")
print("¡PROCESAMIENTO FINALIZADO! Listo para el Paso 5: Entrenamiento del Modelo.")
print("---------------------------------------------------------")

---------------------------------------------------------
INICIO DEL PROCESO MONOLÍTICO: Limpieza, Escalado y SMOTE.
---------------------------------------------------------
1. Cargando archivo: /content/drive/MyDrive/Colab Notebooks/DataAnalytics/iotsim-hydraulic-system-2.csv...
✅ Archivo cargado con éxito. Total de filas: 2114866

2. Codificación de etiquetas (LabelEncoder)...
3. Limpieza y Procesamiento de X...
✅ Limpieza y conversión completada.

4. División y Escalado de Datos...
✅ Split y Escalado completados.

5. Aplicando SMOTE (Balanceo) para la clase minoritaria 'Benign'...
✅ SMOTE aplicado con éxito.

Distribución de Clases DESPUÉS del balanceo (y_train_res):


,count
label_encoded,
9,572278
6,572278
7,572278
8,572278
11,572278
12,572278
3,572278
0,572278
5,572278



---------------------------------------------------------
¡PROCESAMIENTO FINALIZADO! Listo para el Paso 5: Entrenamiento del Modelo.
---------------------------------------------------------


# **PASO 5: ENTRENAMIENTO Y EVALUACIÓN DEL MODELO RANDOM FOREST**

In [5]:
# =========================================================
# PASO 5: ENTRENAMIENTO Y EVALUACIÓN DEL MODELO RANDOM FOREST
# =========================================================
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import time

print("---------------------------------------------------------")
print("INICIO DEL PASO 5: Entrenamiento del Modelo.")
print("---------------------------------------------------------")

# 5.1 Definición del Modelo
# n_jobs=-1 en RandomForestClassifier SÍ está permitido y acelera el proceso.
model = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    class_weight='balanced',
    n_jobs=-1
)

# 5.2 Entrenamiento del Modelo
print("Entrenando el modelo Random Forest (esto puede tomar varios minutos)...")
start_time = time.time() # Medimos el tiempo de entrenamiento

# El modelo se entrena con los datos balanceados (X_train_res, y_train_res)
model.fit(X_train_res, y_train_res)

end_time = time.time()
training_time = end_time - start_time
print(f"✅ Entrenamiento completado en {training_time:.2f} segundos.")


# 5.3 Predicción en el Conjunto de Prueba
# Las predicciones se hacen sobre el conjunto de prueba NO balanceado (X_test)
print("\nHaciendo predicciones sobre el conjunto de prueba...")
y_pred = model.predict(X_test)
print("✅ Predicciones realizadas.")


# 5.4 Evaluación del Modelo
print("\n--- INFORME DE EVALUACIÓN ---")

# Obtenemos los nombres reales de las clases (ej. 'Benign', 'Mirai TCP Flooding')
# usando el LabelEncoder guardado.
target_names = le.classes_

# Informe de Clasificación (métricas clave)
print("\nInforme de Clasificación:")
print(classification_report(y_test, y_pred, target_names=target_names))

# Matriz de Confusión (para ver errores por clase)
print("\nMatriz de Confusión:")
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=target_names, columns=target_names)
print("---------------------------------------------------------")
display(cm_df)
print("---------------------------------------------------------")

print("\n---------------------------------------------------------")
print("¡PROCESO DE MODELADO Y EVALUACIÓN FINALIZADO!")
print("---------------------------------------------------------")

---------------------------------------------------------
INICIO DEL PASO 5: Entrenamiento del Modelo.
---------------------------------------------------------
Entrenando el modelo Random Forest (esto puede tomar varios minutos)...
✅ Entrenamiento completado en 775.60 segundos.

Haciendo predicciones sobre el conjunto de prueba...
✅ Predicciones realizadas.

--- INFORME DE EVALUACIÓN ---

Informe de Clasificación:
                       precision    recall  f1-score   support

               Benign       1.00      1.00      1.00      1634
    C&C Communication       0.55      1.00      0.71        16
        File Download       1.00      1.00      1.00       237
Ingress Tool Transfer       1.00      1.00      1.00       736
 Merlin ICMP Flooding       0.99      1.00      1.00      1959
  Merlin TCP Flooding       0.99      1.00      1.00      4000
  Merlin UDP Flooding       0.01      1.00      0.02      1000
   Mirai GRE Flooding       1.00      1.00      1.00    105335
   Mirai TCP 

,Benign,C&C Communication,File Download,Ingress Tool Transfer,Merlin ICMP Flooding,Merlin TCP Flooding,Merlin UDP Flooding,Mirai GRE Flooding,Mirai TCP Flooding,Mirai UDP Flooding,Reporting,TCP Scan,Telnet Brute Force,Unknown
Benign,1634,0,0,0,0,0,0,0,0,0,0,0,0,0
C&C Communication,0,16,0,0,0,0,0,0,0,0,0,0,0,0
File Download,0,0,237,0,0,0,0,0,0,0,0,0,0,0
Ingress Tool Transfer,0,0,0,736,0,0,0,0,0,0,0,0,0,0
Merlin ICMP Flooding,0,0,0,0,1959,0,0,0,0,0,0,0,0,0
Merlin TCP Flooding,0,0,0,0,0,3999,0,0,1,0,0,0,0,0
Merlin UDP Flooding,0,0,0,0,0,0,1000,0,0,0,0,0,0,0
Mirai GRE Flooding,0,0,0,0,0,0,0,105335,0,0,0,0,0,0
Mirai TCP Flooding,0,0,0,0,0,23,0,0,143047,0,0,0,0,0
Mirai UDP Flooding,0,0,0,0,10,0,80255,0,0,56513,0,0,0,0


---------------------------------------------------------

---------------------------------------------------------
¡PROCESO DE MODELADO Y EVALUACIÓN FINALIZADO!
---------------------------------------------------------
